# Question Recomendation

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [293]:
performance=pd.read_csv("performance.csv", sep=',')
performance

,student_id,class,subtopic_id,quiz_id,question_id,question_level,marked,total_marks,score,score_pct,topic_id,subtopic_name,subject_id
0,17,8.0,574,b16d8de5-9c9a-43b8-b92a-e6c27bbbcde5,5773,remember,0,1,0,0.00,288,Place Value And Total Value,2
1,17,8.0,574,d82aeb1d-7b23-42c9-9ed3-4942de399982,5773,remember,0,1,0,0.00,288,Place Value And Total Value,2
2,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5773,remember,1,6,3,0.50,288,Place Value And Total Value,2
3,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5774,understand,1,6,3,0.50,288,Place Value And Total Value,2
4,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5775,evaluate,1,6,3,0.50,288,Place Value And Total Value,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7703,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6385,evaluate,1,12,8,0.67,176,Signs Of Ill Health In Livestock,3
7704,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6386,evaluate,0,12,8,0.67,176,Signs Of Ill Health In Livestock,3
7705,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6387,evaluate,0,12,8,0.67,176,Signs Of Ill Health In Livestock,3
7706,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6388,evaluate,0,12,8,0.67,176,Signs Of Ill Health In Livestock,3


In [5]:
performance['total_marks'].describe()


count    7708.000000
mean        8.281266
std         4.103472
min         1.000000
25%         5.000000
50%         8.000000
75%        11.000000
max        19.000000
Name: total_marks, dtype: float64

In [8]:
quiz_threshold = 5
quiz_data_balanced_questions = performance.query('total_marks >= @quiz_threshold')
quiz_data_balanced_questions

,student_id,class,subtopic_id,quiz_id,question_id,question_level,marked,total_marks,score,score_pct,topic_id,subtopic_name,subject_id
2,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5773,remember,1,6,3,0.50,288,Place Value And Total Value,2
3,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5774,understand,1,6,3,0.50,288,Place Value And Total Value,2
4,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5775,evaluate,1,6,3,0.50,288,Place Value And Total Value,2
5,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5776,evaluate,0,6,3,0.50,288,Place Value And Total Value,2
6,630,8.0,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,5777,understand,0,6,3,0.50,288,Place Value And Total Value,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6384,remember,0,12,8,0.67,176,Signs Of Ill Health In Livestock,3
7703,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6385,evaluate,1,12,8,0.67,176,Signs Of Ill Health In Livestock,3
7704,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6386,evaluate,0,12,8,0.67,176,Signs Of Ill Health In Livestock,3
7705,29184,8.0,353,42c2c077-0a16-4f0b-bc13-58bcf0da894d,6387,evaluate,0,12,8,0.67,176,Signs Of Ill Health In Livestock,3


In [263]:
quiz_data= quiz_data_balanced_questions.groupby(['student_id','topic_id', 'subtopic_id', 'quiz_id','score','total_marks'])['score_pct'].mean()
quiz_data

student_id  topic_id  subtopic_id  quiz_id                               score  total_marks
17          288       577          aded26aa-3225-457c-9ea2-7d8f186cb7ed  0      6              0.00
                      578          24be5aae-4b5f-4279-8ec8-ca0c689778d2  4      11             0.36
630         288       574          32b70b5a-1efe-4ddd-b20e-7119977c20d1  3      6              0.50
                                   92fa7ef4-61d8-477e-89f8-48e68aae80fa  4      10             0.40
                      575          b5032f60-3330-4607-9c71-8a517bd4d298  3      6              0.50
                                                                                               ... 
31671       293       603          631ea629-9be9-4e17-b88b-bff2e270e793  2      6              0.33
                                   93af2886-29f5-4db9-8125-5362c0b65969  4      6              0.67
33303       294       604          64bb505e-7b04-4d3d-96d4-21554ce6237b  7      7              1.00
        

In [264]:

quiz_data = quiz_data.reset_index()
quiz_data

,student_id,topic_id,subtopic_id,quiz_id,score,total_marks,score_pct
0,17,288,577,aded26aa-3225-457c-9ea2-7d8f186cb7ed,0,6,0.00
1,17,288,578,24be5aae-4b5f-4279-8ec8-ca0c689778d2,4,11,0.36
2,630,288,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,3,6,0.50
3,630,288,574,92fa7ef4-61d8-477e-89f8-48e68aae80fa,4,10,0.40
4,630,288,575,b5032f60-3330-4607-9c71-8a517bd4d298,3,6,0.50
...,...,...,...,...,...,...,...
678,31671,293,603,631ea629-9be9-4e17-b88b-bff2e270e793,2,6,0.33
679,31671,293,603,93af2886-29f5-4db9-8125-5362c0b65969,4,6,0.67
680,33303,294,604,64bb505e-7b04-4d3d-96d4-21554ce6237b,7,7,1.00
681,33303,294,606,d9be3b77-db2f-4cc5-abb9-b9d94fc98ae3,2,5,0.40


In [265]:
quiz_data["Performance_cat"] = pd.cut(quiz_data["score_pct"], bins=[-np.inf, .4, .99, 1], labels=["low", "mid", "high"])
# quiz_data.drop(["Performance Groups","performance_groups","Performance_level_code"],axis=1,inplace=True)

quiz_data

,student_id,topic_id,subtopic_id,quiz_id,score,total_marks,score_pct,Performance_cat
0,17,288,577,aded26aa-3225-457c-9ea2-7d8f186cb7ed,0,6,0.00,low
1,17,288,578,24be5aae-4b5f-4279-8ec8-ca0c689778d2,4,11,0.36,low
2,630,288,574,32b70b5a-1efe-4ddd-b20e-7119977c20d1,3,6,0.50,mid
3,630,288,574,92fa7ef4-61d8-477e-89f8-48e68aae80fa,4,10,0.40,low
4,630,288,575,b5032f60-3330-4607-9c71-8a517bd4d298,3,6,0.50,mid
...,...,...,...,...,...,...,...,...
678,31671,293,603,631ea629-9be9-4e17-b88b-bff2e270e793,2,6,0.33,low
679,31671,293,603,93af2886-29f5-4db9-8125-5362c0b65969,4,6,0.67,mid
680,33303,294,604,64bb505e-7b04-4d3d-96d4-21554ce6237b,7,7,1.00,high
681,33303,294,606,d9be3b77-db2f-4cc5-abb9-b9d94fc98ae3,2,5,0.40,low


In [146]:
cols = list(quiz_data.columns.values)
cols.pop(cols.index('topic_id'))
quiz_data = quiz_data[cols+['topic_id']]
cols = list(quiz_data.columns.values)
cols.pop(cols.index('subtopic_id'))
quiz_data = quiz_data[cols+['subtopic_id']]
cols = list(quiz_data.columns.values)
cols.pop(cols.index('score_pct'))
quiz_data = quiz_data[cols+['score_pct']]
cols = list(quiz_data.columns.values)
cols.pop(cols.index('Performance_cat'))
quiz_data = quiz_data[cols+['Performance_cat']]
quiz_data

,index,student_id,quiz_id,total_marks,topic_id,subtopic_id,score_pct,Performance_cat
0,0,17,aded26aa-3225-457c-9ea2-7d8f186cb7ed,6,288,577,0.00,low
1,1,17,24be5aae-4b5f-4279-8ec8-ca0c689778d2,11,288,578,0.36,low
2,2,630,32b70b5a-1efe-4ddd-b20e-7119977c20d1,6,288,574,0.50,mid
3,3,630,92fa7ef4-61d8-477e-89f8-48e68aae80fa,10,288,574,0.40,low
4,4,630,b5032f60-3330-4607-9c71-8a517bd4d298,6,288,575,0.50,mid
...,...,...,...,...,...,...,...,...
678,678,31671,631ea629-9be9-4e17-b88b-bff2e270e793,6,293,603,0.33,low
679,679,31671,93af2886-29f5-4db9-8125-5362c0b65969,6,293,603,0.67,mid
680,680,33303,64bb505e-7b04-4d3d-96d4-21554ce6237b,7,294,604,1.00,high
681,681,33303,d9be3b77-db2f-4cc5-abb9-b9d94fc98ae3,5,294,606,0.40,low


In [147]:
quiz_data['Performance_cat'] = quiz_data['Performance_cat'].astype('category')

performance_level_dict={'low':1,'mid':2,'high':3 }
quiz_data["performance_cat_code"] = quiz_data.Performance_cat.map(performance_level_dict)
quiz_data.head()

,index,student_id,quiz_id,total_marks,topic_id,subtopic_id,score_pct,Performance_cat,performance_cat_code
0,0,17,aded26aa-3225-457c-9ea2-7d8f186cb7ed,6,288,577,0.00,low,1
1,1,17,24be5aae-4b5f-4279-8ec8-ca0c689778d2,11,288,578,0.36,low,1
2,2,630,32b70b5a-1efe-4ddd-b20e-7119977c20d1,6,288,574,0.50,mid,2
3,3,630,92fa7ef4-61d8-477e-89f8-48e68aae80fa,10,288,574,0.40,low,1
4,4,630,b5032f60-3330-4607-9c71-8a517bd4d298,6,288,575,0.50,mid,2


In [239]:
quiz_data.to_csv(r'/Users/faithtoo/Downloads/quiz_data.csv', index = False, header=True)


In [15]:
questions_data= quiz_data_balanced_questions.groupby(['student_id','topic_id', 'subtopic_id', 'question_id','question_level',])['marked'].mean()
questions_data = questions_data.reset_index()
questions_data

,student_id,topic_id,subtopic_id,question_id,question_level,marked
0,17,288,577,5818,evaluate,0.0
1,17,288,577,5819,evaluate,0.0
2,17,288,577,5820,evaluate,0.0
3,17,288,577,5821,evaluate,0.0
4,17,288,577,5822,evaluate,0.0
...,...,...,...,...,...,...
4202,33396,288,574,5773,remember,1.0
4203,33396,288,574,5774,understand,1.0
4204,33396,288,574,5775,evaluate,0.0
4205,33396,288,574,5776,evaluate,1.0


In [17]:

questions_data['marked'] = questions_data['marked'].astype('int')
questions_data


,student_id,topic_id,subtopic_id,question_id,question_level,marked
0,17,288,577,5818,evaluate,0
1,17,288,577,5819,evaluate,0
2,17,288,577,5820,evaluate,0
3,17,288,577,5821,evaluate,0
4,17,288,577,5822,evaluate,0
...,...,...,...,...,...,...
4202,33396,288,574,5773,remember,1
4203,33396,288,574,5774,understand,1
4204,33396,288,574,5775,evaluate,0
4205,33396,288,574,5776,evaluate,1


In [84]:
questions_data.to_csv(r'/Users/faithtoo/Downloads/questions_data.csv', index = False, header=True)


In [21]:
questions_data['question_level'] = questions_data['question_level'].astype('category')

question_level_dict={'remember':1,'understand':2,'apply':3 ,'analyze':4,'evaluate':5 ,'create':6}
questions_data["question_level_code"] = questions_data.question_level.map(question_level_dict)
questions_data.head()

,student_id,topic_id,subtopic_id,question_id,question_level,marked,question_level_code
0,17,288,577,5818,evaluate,0,5
1,17,288,577,5819,evaluate,0,5
2,17,288,577,5820,evaluate,0,5
3,17,288,577,5821,evaluate,0,5
4,17,288,577,5822,evaluate,0,5


In [157]:
if not questions_data[questions_data.duplicated(['student_id', 'question_id'])].empty:
    initial_rows = questions_data.shape[0]

    print('Initial dataframe shape {0}'.format(questions_data.shape))
    questions_data = questions_data.drop_duplicates(['student_id', 'subtopic_id'])
    current_rows = questions_data.shape[0]
    print ('New dataframe shape {0}'.format(questions_data.shape))
    print( 'Removed {0} rows'.format(initial_rows - current_rows))


In [158]:
questions_data.head()

,student_id,topic_id,subtopic_id,question_id,question_level,marked,question_level_code
0,17,288,577,5818,evaluate,0,5
6,17,288,578,5824,evaluate,1,5
17,630,288,574,5773,remember,1,1
27,630,288,575,5783,understand,0,2
33,630,293,602,6319,evaluate,0,5


In [73]:
wide_questions_data =questions_data .pivot(index = 'question_id', columns = ['student_id','topic_id','subtopic_id'], values = 'marked').fillna(0)
wide_questions_data_sparse = csr_matrix(wide_questions_data.values)

wide_questions_data_sparse


<380x442 sparse matrix of type '<class 'numpy.float64'>'
	with 2465 stored elements in Compressed Sparse Row format>

In [74]:


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(wide_questions_data_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [75]:
query_index = np.random.choice(wide_questions_data.shape[0])
distances, indices = model_knn.kneighbors(wide_questions_data.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print ('Recommendations for {0}:\n'.format(wide_questions_data.index[query_index]))
    else:
        print ('{0}: {1}, with distance of {2}:'.format(i, wide_questions_data.index[indices.flatten()[i]], distances.flatten()[i]))
        
        
        

Recommendations for 5806:

1: 5810, with distance of 0.0:
2: 5809, with distance of 0.2254033307585166:
3: 5808, with distance of 0.2928932188134523:
4: 5811, with distance of 0.33333333333333315:
5: 5807, with distance of 0.3453463292920228:


In [135]:
question = questions_data.loc[(questions_data['question_id'] == 5807 )]
question

,student_id,topic_id,subtopic_id,question_id,question_level,marked,question_level_code
446,11643,288,576,5807,evaluate,0,5
850,11695,288,576,5807,evaluate,0,5
1013,11701,288,576,5807,evaluate,0,5
1132,11705,288,576,5807,evaluate,0,5
1204,11709,288,576,5807,evaluate,0,5
1246,11710,288,576,5807,evaluate,1,5
1427,11732,288,576,5807,evaluate,1,5
1860,17056,288,576,5807,evaluate,1,5
2432,21265,288,576,5807,evaluate,1,5
2897,26429,288,576,5807,evaluate,1,5


# Subtopic Recomendation

In [163]:
data = pd.read_csv("questions_data.csv")
data.head()

,student_id,topic_id,subtopic_id,question_id,question_level,marked,question_level_code
0,17,288,577,5818,evaluate,0,5
1,17,288,577,5819,evaluate,0,5
2,17,288,577,5820,evaluate,0,5
3,17,288,577,5821,evaluate,0,5
4,17,288,577,5822,evaluate,0,5


In [165]:
if not data[data.duplicated(['student_id', 'subtopic_id'])].empty:
    initial_rows = data.shape[0]

    print('Initial dataframe shape {0}'.format(data.shape))
    data = data.drop_duplicates(['student_id', 'subtopic_id'])
    current_rows = data.shape[0]
    print ('New dataframe shape {0}'.format(data.shape))
    print( 'Removed {0} rows'.format(initial_rows - current_rows))

Initial dataframe shape (4014, 7)
New dataframe shape (442, 7)
Removed 3572 rows


In [208]:
wide_quiz_data =data.pivot(index = 'subtopic_id', columns = ['student_id','topic_id'], values = 'marked').fillna(0)
wide_quiz_data_sparse = csr_matrix(wide_quiz_data.values)

wide_quiz_data_sparse

<59x364 sparse matrix of type '<class 'numpy.float64'>'
	with 322 stored elements in Compressed Sparse Row format>

In [209]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(wide_quiz_data_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [225]:

query_index = np.random.choice(wide_quiz_data.shape[0])
distances, indices = model_knn.kneighbors(wide_quiz_data.iloc[query_index, :].values.reshape(1, -1), n_neighbors =5)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print ('Recommendations for {0}:\n'.format(wide_quiz_data.index[query_index]))
    else:
        print ('{0}: {1}, with distance of {2}:'.format(i, wide_quiz_data.index[indices.flatten()[i]], distances.flatten()[i]))
        
        
        

Recommendations for 620:

1: 622, with distance of 0.0:
2: 616, with distance of 0.29289321881345254:
3: 618, with distance of 0.5527864045000421:
4: 611, with distance of 1.0:


In [221]:
subtopic = data.loc[(data['subtopic_id'] == 609 )]
subtopic

,student_id,topic_id,subtopic_id,question_id,question_level,marked,question_level_code
1869,17062,294,609,6368,evaluate,0,5
3659,30395,294,609,6368,evaluate,1,5


#  Student Classification

In [292]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from seaborn import load_dataset
from sklearn.model_selection import train_test_split


quiz_data = pd.read_csv("quiz_data.csv")
# Store features matrix in X
X= quiz_data.iloc[:, 6:7].values
#Store target vector in 
y= quiz_data.iloc[:, 7].values

# Splitting data into training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100)

KNeighborsClassifier(
    n_neighbors=7,          # The number of neighbours to consider
    weights='uniform',      # How to weight distances
    algorithm='auto',       # Algorithm to compute the neighbours
    leaf_size=30,           # The leaf size to speed up searches
    p=2,                    # The power parameter for the Minkowski metric
    metric='minkowski',     # The type of distance to use
    metric_params=None,     # Keyword arguments for the metric function
    n_jobs=None             # How many parallel jobs to run
)

# Creating a classifier object in sklearn
clf = KNeighborsClassifier(p=1)

# Fitting our model
clf.fit(X_train, y_train)

# Making predictions
predictions = clf.predict(X_test)

# Making your own predictions
ith_prediction = clf.predict([[0.80]])
print(ith_prediction)

# Measuring the accuracy of our model
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))



 

['mid']
1.0
